# Examples - Array - Alternating Least Squares for collaborative filtering
https://gist.github.com/mrocklin/017f78ce52d265b6d72828fb29e5619c

## Tensor Decompositions with NumPy and Dask.array
We implement a very simple algorithm to compute the PARAFAC/CANDECOMP tensor decomposition using alternating least squares.

We follow section 4.1 from this paper: https://www.cs.cmu.edu/~pmuthuku/mlsp_page/lectures/Parafac.pdf

PARAFAC/CANDECOMP is an array decomposition that generalizes SVD/PCA to higher dimensions. It is commonly used when trying to find low-rank structure in observational data in multiple dimensions.

<img src='https://camo.githubusercontent.com/a6e062883b83adb3b65b5a9e167a3a6f5e5f9a19/68747470733a2f2f75706c6f61642e77696b696d656469612e6f72672f77696b6970656469612f636f6d6d6f6e732f352f35322f436f6c6c61626f7261746976655f66696c746572696e672e676966'>

In a typical two dimensional setting we might call this collaborative filtering, and is an approach used by companies like Amazon and Netflix to help recommend products to users based on the habits of similar users. However nothing about this problem is specifically restricted to two dimensions. We might consider the following applications:

- What customers bought what products at what times of day or year
- Which computers talked to which other computers over which ports under which user
- Outcomes of treatments affecting patients with which dieseases

When we look around for algorithms to generalize SVD/PCA to multiple dimensions we come across a family of algorithms called **tensor decompositions**. This notebook implements the simplest such algorithm in a common and simple way using NumPy and then scales it out naively using Dask arrays.

### Alternating Least Squares

In [1]:
import numpy as np
from functools import reduce
import operator
import dask


def outer_product(*Xs):
    """ Outer/tensor product of multiple 2d arrays

    Parameters
    ----------
    *Xs: arrays of size (k, n)
        All inputs must have the same first dimension but may have varying
        second dimensions
    """
    n = len(Xs)
    indexes = [(slice(None, None),) +
               (None,) * i +
               (slice(None, None),) +
               (None,) * (n - i - 1) for i in range(len(Xs))]
    Ys = [X[ind] for X, ind in zip(Xs, indexes)]
    Ys = sorted(Ys, key=lambda y: y.nbytes)  # smaller outer products first
    return reduce(operator.mul, Ys)


def parafac_als(X, n_factors, n_iter=100):
    """ Parafac tensor decomposition

    This implements the basic algorithm in section 4.1 of this paper

        https://www.cs.cmu.edu/~pmuthuku/mlsp_page/lectures/Parafac.pdf
    """
    # Randomly initialize factors
    factors = [np.random.random((n_factors, X.shape[i])) for i in range(0, X.ndim)]
    
    # Solve
    for itr in range(n_iter):
        for i in range(X.ndim):
            not_i = tuple(j for j in range(X.ndim) if j != i)
            Xp = X.transpose((i,) + not_i)
            Xp = Xp.reshape((Xp.shape[0], np.prod(Xp.shape[1:])))
            Z = outer_product(*[factors[j] for j in not_i])
            Z = Z.reshape((Z.shape[0], np.prod(Z.shape[1:])))

            factor, residuals, rank, s = np.linalg.lstsq(Z.T, Xp.T)
            factors[i] = factor
    return factors

### Demonstrate accuracy

In [2]:
true_factors = [[[1, 1, 1, 0, 0, 0],
                 [1, 0, 1, 0, 1, 0]],
                [[1, 0],
                 [1, 1]],
                [[0, 1],
                 [1, 1]],
                [[0, 0, 1, 1, 0],
                 [1, 1, 0, 1, 1]]]
true_factors = [np.array(x).reshape((2, len(x[0]))) for x in true_factors]

X = outer_product(*true_factors).sum(axis=0)
X

array([[[[1, 1, 0, 1, 1],
         [1, 1, 1, 2, 1]],

        [[1, 1, 0, 1, 1],
         [1, 1, 0, 1, 1]]],


       [[[0, 0, 0, 0, 0],
         [0, 0, 1, 1, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[1, 1, 0, 1, 1],
         [1, 1, 1, 2, 1]],

        [[1, 1, 0, 1, 1],
         [1, 1, 0, 1, 1]]],


       [[[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[1, 1, 0, 1, 1],
         [1, 1, 0, 1, 1]],

        [[1, 1, 0, 1, 1],
         [1, 1, 0, 1, 1]]],


       [[[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]]])

In [3]:
factors = parafac_als(X, 3, 10)

computed = outer_product(*factors).sum(axis=0)
computed

/home/wei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


array([[[[ 9.94701717e-01,  9.94701717e-01,  1.90151086e-04,
           9.94891868e-01,  9.94701717e-01],
         [ 9.98483286e-01,  9.98483286e-01,  1.01228942e+00,
           2.01077271e+00,  9.98483286e-01]],

        [[ 1.00206577e+00,  1.00206577e+00, -3.23840338e-03,
           9.98827364e-01,  1.00206577e+00],
         [ 9.94457318e-01,  9.94457318e-01,  1.59216427e-02,
           1.01037896e+00,  9.94457318e-01]]],


       [[[-5.72293108e-03, -5.72293108e-03,  4.69442356e-03,
          -1.02850751e-03, -5.72293108e-03],
         [ 1.27144533e-02,  1.27144533e-02,  9.29383638e-01,
           9.42098092e-01,  1.27144533e-02]],

        [[-4.38539968e-03, -4.38539968e-03, -4.97641313e-03,
          -9.36181281e-03, -4.38539968e-03],
         [-3.12551541e-02, -3.12551541e-02,  1.36705758e-01,
           1.05450603e-01, -3.12551541e-02]]],


       [[[ 9.94701717e-01,  9.94701717e-01,  1.90151086e-04,
           9.94891868e-01,  9.94701717e-01],
         [ 9.98483286e-01,  9.9848

In [4]:
computed.round(0) - X

array([[[[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.]],

        [[ 0.,  0., -0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.]]],


       [[[-0., -0.,  0., -0., -0.],
         [ 0.,  0.,  0.,  0.,  0.]],

        [[-0., -0., -0., -0., -0.],
         [-0., -0.,  0.,  0., -0.]]],


       [[[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.]],

        [[ 0.,  0., -0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.]]],


       [[[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.]]],


       [[[ 0.,  0., -0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0., -0.,  0.,  0.]]],


       [[[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.]]]])

### Scale with Dask.array
The ALS algorithm above is largely reshapings, scalar multiplication, matrix multiplication, and least squares computations (A QR decomposition followed by a triangular solve) all of which Dask.array can do easily. Here we adapt our code from before to work on dask.arrays. We include the original unchanged lines to show how easy it is to adapt.

In [5]:
import dask.array as da

def parafac_als(X, n_factors, n_iter=100):
    """ Parafac tensor decomposition

    This implements the basic algorithm in section 4.1 of this paper

        https://www.cs.cmu.edu/~pmuthuku/mlsp_page/lectures/Parafac.pdf
    """
    # Randomly initialize factors
    # factors = [np.random.random((n_factors, X.shape[i])) for i in range(0, X.ndim)]
    factors = [da.random.random((n_factors, X.shape[i]), 
                                chunks=(None, X.chunks[i])) 
               for i in range(0, X.ndim)]
    
    # Solve
    for itr in range(n_iter):
        for i in range(X.ndim):
            not_i = tuple(j for j in range(X.ndim) if j != i)
            Xp = X.transpose((i,) + not_i)
            Xp = Xp.reshape((Xp.shape[0], np.prod(Xp.shape[1:])))
            Z = outer_product(*[factors[j] for j in not_i])
            Z = Z.reshape((Z.shape[0], np.prod(Z.shape[1:])))

            # factor, residuals, rank, s = np.linalg.lstsq(Z.T, Xp.T)
            factor, residuals, rank, s = da.linalg.lstsq(Z.T, Xp.T)
            factors[i] = factor
            
    return factors

#### Visualize result
To show the complexity of the algorithm we visualize a single round of chunked ALS as a task graph

In [6]:
dX = da.from_array(X, chunks=((2, 2, 2, 2)))
factors = parafac_als(dX, 2, 1)

# dask.visualize(*factors)

/home/wei/anaconda3/lib/python3.6/site-packages/dask/array/linalg.py:791: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  np.array([0, 1], dtype=b.dtype))


#### Verify accuracy
We can compare against the numpy version to verify that we continue to get the same result.

In [7]:

import dask

factors = parafac_als(dX, 2, 10)

computed = outer_product(*factors).sum(axis=0).compute()
computed

/home/wei/anaconda3/lib/python3.6/site-packages/dask/array/linalg.py:791: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  np.array([0, 1], dtype=b.dtype))


array([[[[ 0.97573206,  0.97573206,  0.03464583,  1.01037789,
           0.97573206],
         [ 0.99723333,  0.99723333,  1.01331671,  2.01055004,
           0.99723333]],

        [[ 1.01702812,  1.01702812, -0.0470655 ,  0.96996262,
           1.01702812],
         [ 0.98032032,  0.98032032,  0.03852199,  1.01884232,
           0.98032032]]],


       [[[-0.03196896, -0.03196896,  0.07780312,  0.04583417,
          -0.03196896],
         [ 0.02343247,  0.02343247,  0.93261202,  0.95604449,
           0.02343247]],

        [[-0.03809417, -0.03809417,  0.00853454, -0.02955963,
          -0.03809417],
         [-0.03190624, -0.03190624,  0.0814083 ,  0.04950205,
          -0.03190624]]],


       [[[ 0.97573206,  0.97573206,  0.03464583,  1.01037789,
           0.97573206],
         [ 0.99723333,  0.99723333,  1.01331671,  2.01055004,
           0.99723333]],

        [[ 1.01702812,  1.01702812, -0.0470655 ,  0.96996262,
           1.01702812],
         [ 0.98032032,  0.98032032,  0.0

In [8]:
(computed - X).round(0)

array([[[[-0., -0.,  0.,  0., -0.],
         [-0., -0.,  0.,  0., -0.]],

        [[ 0.,  0., -0., -0.,  0.],
         [-0., -0.,  0.,  0., -0.]]],


       [[[-0., -0.,  0.,  0., -0.],
         [ 0.,  0., -0., -0.,  0.]],

        [[-0., -0.,  0., -0., -0.],
         [-0., -0.,  0.,  0., -0.]]],


       [[[-0., -0.,  0.,  0., -0.],
         [-0., -0.,  0.,  0., -0.]],

        [[ 0.,  0., -0., -0.,  0.],
         [-0., -0.,  0.,  0., -0.]]],


       [[[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.]]],


       [[[ 0.,  0., -0., -0.,  0.],
         [-0., -0.,  0.,  0., -0.]],

        [[ 0.,  0., -0., -0.,  0.],
         [ 0.,  0., -0., -0.,  0.]]],


       [[[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.]]]])

### Conclusions
This notebook provides two contributions:

- A simple alternating least squares implementation of a simple array decomposition
- A demonstration of the ability of Dask.array to easily parallelize existing NumPy algorithms

Future work on this notebook could include:

- Improving the algorithm with better initialization conditions or stopping criteria
- Performing scaling and benchmarking experiments to study how the parallel/distributed version performs